In [ ]:
import numpy as np
import pandas as pd
import re

In [ ]:
eat_2018 = pd.read_csv(f"{address}즉석섭취식품_2018_2020.csv", encoding = "utf-8")
eat_2021 = pd.read_csv(f"{address}즉석섭취식품_2021_2022.csv", encoding = "utf-8")
cook_2018 = pd.read_csv(f"{address}즉석조리식품_2018_2020.csv", encoding = "utf-8")
cook_2021 = pd.read_csv(f"{address}즉석조리식품_2021_2022.csv", encoding = "utf-8")
fresh_2018 = pd.read_csv(f"{address}신선편의식품_2018_2020.csv", encoding = "utf-8")
fresh_2021 = pd.read_csv(f"{address}신선편의식품_2021_2022.csv", encoding = "utf-8")
member = pd.read_csv(f"{address}전체가구수.csv", encoding = "utf-8")
member_1 = pd.read_csv(f"{address}1인가구.csv")
reason_2018 = pd.read_csv(f"{address}간편식_HMR__구입_이유_2018.csv", encoding = "utf-8")
reason_2022 = pd.read_csv(f"{address}간편식_HMR__구입_이유_2019-2022.csv", encoding = "utf-8")
consume = pd.read_csv(f"{address}간편식_지출액.csv", encoding='cp949')
check_list = pd.read_csv(f"{address}간편식_구입시_확인사항.csv", encoding='cp949')

# 함수 및 리스트

In [ ]:
# 재배열 위한 리스트
new_col_list2021 = ['year', "항목", '소계', '수도권', '충청권', '호남권', '대경권', '동남권', '강원권', '1인', '2인', '3인', '4인이상', '남성', '여성', '20대', '30대', '40대', '50대', '60대이상',
                    '200만원미만', '200~400만원미만', '400~600만원미만','600만원이상']
# 즉시섭취식품 맵핑 dict
eat_dict = {
    "샐러드" : "즉석섭취반찬류",
    "반찬류" : "즉석섭취반찬류",
    "도시락" : "즉석섭취식사류",
    "김밥류" : "즉석섭취식사류",
    "샌드위치/햄버거류" : "즉석섭취식사류"
}
# 즉시조리식품 맵핑 dict
cook_dict = {
    "밥류" : "즉석밥류",
    "면류" : "즉석떡볶이/면류",
    "국류" : "즉석국류/찌개/탕류",
    "찌개/탕류" : "즉석국류/찌개/탕류",
    "죽/스프류" : "죽/스프류",
    "육류" : "육류",
    "수산물" : "수산물",
    "만두/피자류" : "만두/피자류",
    "소스류" : "소스/양념류",
    "앙념류" : "소스/양념류"
}
# 신선편의식품 맵핑 dict
fresh_dict = {
    "신선편의채소" : "신선편의채소",
    "신선편의과일" : "신선편의과일"
}

#
year18_20 = ["2018", "2019", "2020"]
year21_22 = ["2021", "2022"]

cate_2018 = {"즉석섭취식사류" : 3,
        "즉석섭취반찬류" : 2,
        "즉석국류/찌개/탕류" : 2,
        "소스/양념류" : 2
        }
cate_2021 = {
        "신선편의채소": 2
}
# 전체가구수 컬럼 정렬
columns_set = ["year", '행정구역별',
 '전국',
 '읍부',
 '면부',
 '동부',
 '서울특별시',
 '부산광역시',
 '대구광역시',
 '인천광역시',
 '광주광역시',
 '대전광역시',
 '울산광역시',
 '세종특별자치시',
 '경기도',
 '강원도',
 '충청북도',
 '충청남도',
 '전라북도',
 '전라남도',
 '경상북도',
 '경상남도',
 '제주특별자치도']

In [ ]:
# 컬럼 2개 병합하고 나누는 함수
def combine_col(x, y):
    col = round((x + y) / 2, 1)
    return col

In [ ]:
# 중복된 내용 병합 및 value 재설정
def col_splits(data, year, col, num):
    col_drop = data[(data["year"] == year) & (data["항목"].str.contains(col))].index
    grby = data[(data["year"] == year) & (data["항목"].str.contains(col))].groupby(["year", "항목"]).sum().reset_index()
    grby.iloc[:, 2:] = np.round(grby.iloc[:, 2:].values / num, 2)
    return grby, col_drop

In [ ]:
# 18~20 데이터 표준화
def preprocessing2018(raw_df, check_dict):

    pattern = re.compile(r"\S")
    # 행렬 변환
    raw_df = raw_df.T

    # 컬럼 추출
    raw_df_columns = raw_df.loc["특성별(2)"].str.replace(" ", "", regex = True)[3:]

    # 연도 추출
    year = [raw_df.index.values[i].split(".")[0] for i in range(len(raw_df.index))][2:]

    # 인덱스 추출 및 매핑
    raw_df_idx = raw_df[0].map(check_dict) + "_" + raw_df[2]

    # values 추출
    raw_df = raw_df.iloc[2:, 3:].astype(float)

    # 인덱스 특수문자 제거
    raw_df_idx = raw_df_idx[2:].apply(lambda x: "".join(pattern.findall(x.split("(")[0])))

    # 컬럼 적용
    raw_df.columns = raw_df_columns.values

    # 인덱스 적용
    raw_df.index = raw_df_idx

    # 연도 추가
    raw_df["year"] = year

    # 파생변수 생성
    raw_df["4인이상"] = combine_col(raw_df["4인"], raw_df["5인이상"])
    raw_df["200~400만원미만"] = combine_col(raw_df["200~300만원미만"], raw_df["300~400만원미만"])
    raw_df["400~600만원미만"]= combine_col(raw_df["400~500만원미만"], raw_df["500~600만원미만"])
    raw_df.drop(["4인", "5인이상", "200~300만원미만", "300~400만원미만", "400~500만원미만", "500~600만원미만"], axis = 1, inplace = True)

    # 인덱스 초기화
    raw_df.reset_index(names = "항목", inplace = True)

    return raw_df[new_col_list2021]

In [ ]:
# 21~22 데이터 표준화
def preprocessing2021(raw_df):

    pattern = re.compile(r"\S")
    # 행렬 변환
    raw_df = raw_df.T

    # 컬럼 추출
    raw_df_columns = raw_df.loc["특성별(2)"].str.replace(" ", "", regex = True)[3:]

    # 연도 추출
    year = [raw_df.index.values[i].split(".")[0] for i in range(len(raw_df.index))][2:]

    # 18~20년 데이터와 형식 맞추기
    raw_df[2].replace("구입", "", regex = True, inplace = True)

    # 인덱스 추출 및 매핑
    raw_df_idx = raw_df[0] + "_" + raw_df[2]

    # values 추출
    raw_df = raw_df.iloc[2:, 3:].astype(float)

    # 인덱스 특수문자 제거
    raw_df_idx = raw_df_idx[2:].apply(lambda x: "".join(pattern.findall(x.split("(")[0])))

    # 컬럼 적용
    raw_df.columns = raw_df_columns.values

    # 인덱스 적용
    raw_df.index = raw_df_idx

    # 연도 추가
    raw_df["year"] = year

    # 인덱스 초기화
    raw_df.reset_index(names = "항목", inplace = True)

    return raw_df[new_col_list2021]

In [ ]:
# 구입이유 처리
def reason_preprocessing(data):
    data = data.T
    data = data.reset_index()

    # 연도 / 인덱스 / 컬럼 / data 각각 따로 추출
    year = data["index"][2:].apply(lambda x: x.split(".")[0])
    reason_idx = data[0][2:]
    reason_col = data.loc[1][2:].str.replace(" ", "")
    try:
        df = data.iloc[2:, 2:].astype(float)
    except:
        df = data.replace("-", 0, inplace = True)
        df = data.iloc[2:, 2:].astype(float)

    # 인덱스, 컬럼, 연도 적용
    df.columns = reason_col.values
    df["항목"] = reason_idx.values
    df["year"] = year

    df = df.reset_index(drop = True)

    if "4인이상" in df.columns:
        over4 = round((df["4인"] + df["5인이상"] + df["4인이상"]) / 3, 1)
    else:
        over4 = round((df["4인"] + df["5인이상"]) / 2, 1)

    under2_4 = round((df["200~300만원미만"] + df["300~400만원미만"]) / 2, 1)
    under4_6 = round((df['400~500만원미만'] + df['500~600만원미만']) / 2, 1)
    df["4인이상"] = over4
    df["200~400만원미만"] = under2_4
    df["400~600만원미만"] = under4_6
    df = df[new_col_list2021]

    if len(df.index) == 9:
        df.loc[9] = ["2018", "기타"] + [0 for i in range(22)]
    else:
        df.loc[49] = ["2022", "기타"] + [0 for i in range(22)]

    return df

In [ ]:
# 소비량 처리
def consume_preprocessing(data):
    data = data.T
    data = data.reset_index()
    # 연도추출
    year = [data["index"].values[i].split(".")[0] for i in range(len(data.index))][2:]
    # 인덱스 추출
    raw_df_idx = data[0][2:].str.replace(" ", "").values
    # 컬럼 추출
    raw_df_col = data.loc[1][2:].str.replace(" ", "").values
    # value 추출
    df = data.iloc[2:, 2:].applymap(lambda x: x.replace("-", "0")).astype(float)
    df.columns = raw_df_col
    df["항목"] = raw_df_idx
    df["year"] = year
    over4 = round((df["4인"] + df["4인이상"] + df["5인이상"]) / 3, 2)
    under2_4 = round((df["200~300만원미만"] + df["200~400만원미만"] + df["300~400만원미만"]) / 3, 2)
    under4_6 = round((df['400~500만원미만'] + df['400~600만원미만'] + df['500~600만원미만']) / 3, 2)
    df.drop(['4인', '4인이상', '5인이상', '200~300만원미만', '200~400만원미만', '300~400만원미만', '400~500만원미만', '400~600만원미만', '500~600만원미만'], axis = 1, inplace = True)
    df["4인이상"] = over4
    df["200~400만원미만"] = under2_4
    df["400~600만원미만"] = under4_6

    return df[new_col_list2021]

In [ ]:
eat2018 = preprocessing2018(eat_2018, eat_dict)
eat2021 = preprocessing2021(eat_2021)
cook2018 = preprocessing2018(cook_2018, cook_dict)
cook2021 = preprocessing2021(cook_2021)
fresh2018 = preprocessing2018(fresh_2018, fresh_dict)
fresh2021 = preprocessing2021(fresh_2021)

df = pd.concat([eat2018, eat2021, cook2018, cook2021, fresh2018, fresh2021], axis = 0).reset_index(drop = True)
df["항목"] = df["항목"].apply(lambda x: x.replace("샐러드", "신선편의채소"))

temp2018 = []
temp2021 = []
col_drops2018 = []
col_drops2021 = []

for i in cate_2018.items():
    for j in year18_20:
        test, col_drop = col_splits(df, year = j, col = i[0], num = i[1])
        temp2018.append(test)
        col_drops2018.append(col_drop)

for i in cate_2021.items():
    for j in year21_22:
        test, col_drop = col_splits(df, year = j, col = i[0], num = i[1])
        temp2021.append(test)
        col_drops2021.append(col_drop)

total_temp = temp2018 + temp2021
total_col_drops = col_drops2018 + col_drops2021

for i in total_col_drops:
    df.drop(i, axis = 0, inplace = True)

new_df = pd.concat([i for i in total_temp])

total_df = pd.concat([df, new_df], axis = 0).reset_index(drop=True)

total_df.to_csv(f"/content/drive/MyDrive/미드프로젝트/data/구입경험변화.csv", index = False)

In [ ]:
# 행렬변환
member = member.T.reset_index()

# 연도 추출
year = [member["index"].values[i].split(".")[0] for i in range(len(member.index))][1:]

# 인덱스 추출 및 컬럼 이름 변경
member_idx = member[0].replace("일반가구_계", "합계")

# 컬럼 추출
member_cols = member.loc[0][2:]

# values 추출
member = member.iloc[1:, 2:].astype(int)

# 컬럼 적용
member.columns = member_cols

# 인덱스 적용
member["행정구역별"] = member_idx

# 연도 적용
member["year"] = year

# 컬럼 정렬
df = member[columns_set]

# 버릴 idx 추출 및 데이터 정렬
drop_idx = df[df["행정구역별"].str.contains("4인|5인|6인|7인 이상")].index
temp = df[df["행정구역별"].str.contains("4인|5인|6인|7인 이상")]
temp = temp.groupby("year").sum().reset_index()
temp["행정구역별"].replace("4인5인6인7인 이상", "4인이상", inplace = True)
df.drop(drop_idx, inplace = True)
df = pd.concat([df, temp]).reset_index(drop = True)

df.to_csv(f"/content/drive/MyDrive/미드프로젝트/data/전체가구수.csv", index = False)

In [ ]:
# 필요한 컬럼만 추출
member1 = member_1[["행정구역별(시군구)","성별", "2018", "2019", "2020", "2021", "2022"]]

# 필요없는 row 제거
member1.drop(list(range(4, 13)), inplace = True)
member1 = member1[1:]

# 합계만 추출하고 필요없는 컬럼 삭제
member1 = member1[member1["성별"] == "계"].drop("성별", axis = 1).reset_index(drop = True)

# 컬럼 이름 정리
member1.columns = ["시군구", "2018", "2019", "2020", "2021", "2022"]
member1.to_csv(f"/content/drive/MyDrive/미드프로젝트/data/1인가구수_시별.csv", index = False)

In [ ]:
data2018 = reason_preprocessing(reason_2018)
data2022 = reason_preprocessing(reason_2022)
df = pd.concat([data2018, data2022]).reset_index(drop = True)

df.to_csv(f"/content/drive/MyDrive/미드프로젝트/data/구입이유.csv", index = False)

In [ ]:
df = consume_preprocessing(consume)
df.to_csv(f"/content/drive/MyDrive/미드프로젝트/data/소비량.csv", index = False)

In [ ]:
check_list = check_list.T
check_list = check_list.reset_index()
# 연도추출
year = [check_list["index"].values[i].split(".")[0] for i in range(len(check_list.index))][2:]
# 인덱스 추출
raw_df_idx = check_list[0][2:].str.replace(" ", "").values
# 컬럼 추출
raw_df_col = check_list.loc[1][2:].str.replace(" ", "").values
# value 추출
df = check_list.iloc[2:, 2:].applymap(lambda x: x.replace("-", "0")).astype(float)

df.columns = raw_df_col
df["항목"] = raw_df_idx
df["year"] = year
df = df.reset_index(drop = True)

df["비수도권_"] = round((df['비수도권'] + df['호남권'] + df['대경권'] + df['동남권']+ df['강원권']) / 3, 2)
df["2~3인_"] = round((df['2인'] + df['2~3인'] + df['3인']) / 3, 2)
df["4인이상_"]= round((df['4인'] + df['4인이상'] + df['5인이상']) / 3, 2)
df["20~30대_"] = round((df['20대'] + df['20~30대'] + df['30대']) / 3, 2)
df["40~50대_"] = round((df['40대'] + df['40~50대'] + df['50대']) / 3, 2)
df["200~400만원미만_"] = round((df['200~300만원미만'] + df['200~400만원미만'] + df['300~400만원미만']) / 3, 2)
df["400~600만원미만_"] = round((df['400~500만원미만'] + df['400~600만원미만'] + df['500~600만원미만']) / 3, 2)

df = df[['year', '항목', '소계', '수도권', '비수도권_', '1인', '2~3인_', '4인이상_', '남성', '여성',
       '20~30대_', '40~50대_', '60대이상', '200만원미만', '200~400만원미만_',
       '400~600만원미만_', '600만원이상']]

new_row2021 = round(df.iloc[[41, 44, 45, 46], 2:].mean(), 2)
new_row2022 = round(df.iloc[[52, 55, 56], 2:].mean(), 2)

new_row2021["year"] = "2021"
new_row2021["항목"] = "기타"
new_row2022["year"] = "2022"
new_row2022["항목"] = "기타"

new_df = pd.DataFrame([new_row2021, new_row2022], columns = df.columns)

drop_row_list = [41, 44, 45, 46, 52, 55, 56]
df.drop(drop_row_list, inplace = True)
df = pd.concat([df, new_df]).reset_index(drop = True)
df.to_csv("/content/drive/MyDrive/미드프로젝트/data/확인사항.csv", index = False)